In [1]:
import QuanTorch as qtorch
import QuanTorch.basis as qbasis
import QuanTorch.operations as qoperations
import QuanTorch.checkers as qcheckers
import QuanTorch.helpers as qhelpers
import QuanTorch.gates as qgates
from QuanTorch.states import qstate, density_matrix

from math import *
import inspect
from IPython.display import display, Math

In [6]:
plus = qbasis.plus
result = qgates.X(qgates.Z(plus))
result

tensor([[-0.7071+0.j],
        [ 0.7071+0.j]])

## Index:
- [Dirac Notation](#dirac)
    - [Basis Vectors & States](#basis)
    - [Basic Operations](#operations)
    - [Checkers](#checkers)
    - [Density Matrix](#dmatrix)
    - [Finding Probabilities](#probs)
- [Quantum Gates](#gates)
    - [Check if gates are unitary](#unitary)
    - [Single Qubit Gates](#single)
    - [Multi Qubit Gates](#multi)
    - <strike> [Visualization](#Vis)</strike> (#TODO)
- <strike> [Quantum Circuits](#circuits)</strike> (#TODO)



# Dirac Notation
<a id='dirac'></a>


![Dirac image](assets/dirac.png)

## Basis Vectors & States
<a id='basis'></a>

In [2]:
zero = qbasis.zero
print(f"zero: {zero}", end="\n" + "-" * 50 + "\n")
print(
    f"Notice that it's a column vector: Shape is:\n {zero.shape}",
    end="\n" + "-" * 50 + "\n",
)
print(
    f"The default data type supports complex numbers: Data type is:\n {zero.dtype}",
    end="\n" + "-" * 50 + "\n",
)
print(
    f"Similarly, one: {qbasis.one}\n\n plus: {qbasis.plus}\n\n minus: {qbasis.minus}")

zero: tensor([[1.+0.j],
        [0.+0.j]])
--------------------------------------------------
Notice that it's a column vector: Shape is:
 torch.Size([2, 1])
--------------------------------------------------
The default data type supports complex numbers: Data type is:
 torch.complex64
--------------------------------------------------
Similarly, one: tensor([[0.+0.j],
        [1.+0.j]])

 plus: tensor([[0.7071+0.j],
        [0.7071+0.j]])

 minus: tensor([[ 0.7071+0.j],
        [-0.7071+0.j]])


## Basic Operations
<a id='basis'></a>

In [3]:
print("Supported operations:")
operations = [o[0] for o in inspect.getmembers(
    qoperations) if inspect.isfunction(o[1])]
for operation in operations:
    print(f"- {operation}")

Supported operations:
- inner_product
- outer_product
- tensor_product


In [4]:
# inner product
print("inner product:\n")
print(f"< 0 | 0 >:\n {qoperations.inner_product(qbasis.zero, qbasis.zero)}")
print(f"< 0 | 1 >:\n {qoperations.inner_product(qbasis.zero, qbasis.one)}")
print(f"< + | - >:\n {qoperations.inner_product(qbasis.plus, qbasis.minus)}")
print(f"< + | 0 >:\n {qoperations.inner_product(qbasis.plus, qbasis.zero)}")

inner product:

< 0 | 0 >:
 tensor([[1.+0.j]])
< 0 | 1 >:
 tensor([[0.+0.j]])
< + | - >:
 tensor([[0.+0.j]])
< + | 0 >:
 tensor([[0.7071+0.j]])


In [29]:
# outer product
print("\nouter product:\n")
print(f"| 0 >< 0 |:\n {qoperations.outer_product(qbasis.zero, qbasis.zero)}")
print(f"| 0 >< 1 |:\n {qoperations.outer_product(qbasis.zero, qbasis.one)}")
print(f"| + >< - |:\n {qoperations.outer_product(qbasis.plus, qbasis.minus)}")
print(f"| + >< 0 |:\n {qoperations.outer_product(qbasis.plus, qbasis.zero)}")


outer product:

| 0 >< 0 |:
 tensor([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]])
| 0 >< 1 |:
 tensor([[0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j]])
| + >< - |:
 tensor([[ 0.5000+0.j, -0.5000-0.j],
        [ 0.5000+0.j, -0.5000-0.j]])
| + >< 0 |:
 tensor([[0.7071+0.j, 0.0000+0.j],
        [0.7071+0.j, 0.0000+0.j]])


In [6]:
# tensor product
print("tensor product:\n")

print(f"| 0 0 > = :\n {qoperations.tensor_product(qbasis.zero, qbasis.zero)}")
print(f"Shape is {qoperations.tensor_product(qbasis.zero, qbasis.zero).shape}")

print(f"| 0 1 > = :\n {qoperations.tensor_product(qbasis.zero, qbasis.one)}")
print(f"Shape is {qoperations.tensor_product(qbasis.zero, qbasis.one).shape}")

print(f"| + - > = :\n {qoperations.tensor_product(qbasis.plus, qbasis.minus)}")
print(f"Shape is {qoperations.tensor_product(qbasis.plus, qbasis.minus).shape}")

tensor product:

| 0 0 > = :
 tensor([[1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]])
Shape is torch.Size([4, 1])
| 0 1 > = :
 tensor([[0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j]])
Shape is torch.Size([4, 1])
| + - > = :
 tensor([[ 0.5000+0.j],
        [-0.5000+0.j],
        [ 0.5000+0.j],
        [-0.5000+0.j]])
Shape is torch.Size([4, 1])


## Checkers
<a id='checkers'></a>

In [7]:
print("Supported checkers:")
checkers = [o[0] for o in inspect.getmembers(
    qcheckers) if inspect.isfunction(o[1])]
for checker in checkers:
    if checker not in operations:
        print(f"- {checker}")

Supported checkers:
- check_basis
- check_normalized
- check_orthogonal
- check_qubit
- check_unitary
- check_valid_density_matrix
- positive_operator


In [8]:
# check normalizedity
print("Check orthogonality:\n")
print(
    f"Check if | 0 > and | 1 > are orthogonal: {qcheckers.check_orthogonal([qbasis.zero, qbasis.one])}"
)
print(
    f"Check if | 0 > and | + > are orthogonal: {qcheckers.check_orthogonal([qbasis.zero, qbasis.plus])}"
)
print(
    f"Check if | 0 > and | - > are orthogonal: {qcheckers.check_orthogonal([qbasis.zero, qbasis.minus])}"
)
print(
    f"Check if | - > and | + > are orthogonal: {qcheckers.check_orthogonal([qbasis.minus, qbasis.plus])}"
)

Check orthogonality:

Check if | 0 > and | 1 > are orthogonal: True
Check if | 0 > and | + > are orthogonal: False
Check if | 0 > and | - > are orthogonal: False
Check if | - > and | + > are orthogonal: True


In [9]:
# check normality
print("Check normality:\n")
print(
    f"Check if | 0 > is normalized: {qcheckers.check_normalized(qbasis.zero)}")
print(
    f"Check if [-1, 3i, i] is normalized: {qcheckers.check_normalized(qstate([-1, 3j, 1j]))}"
)
# Sheet 1 | Q3
print(
    f"Check if | Ψ + > is normalized: {qcheckers.check_normalized(qstate([0, 1 / sqrt(2), 1 / sqrt(2), 0]))}"
)

Check normality:

Check if | 0 > is normalized: True
Check if [-1, 3i, i] is normalized: False
Check if | Ψ + > is normalized: True


In [10]:
# Check basis
# Sheet 1 | Q1
print("Check basis (orthonormality):\n")
print(
    f"Check if | 0 > and | 1 > are valis basis: {qcheckers.check_basis([qbasis.zero, qbasis.one])}"
)
print(
    f"Check if | + > and | - > are valis basis: {qcheckers.check_basis([qbasis.plus, qbasis.minus])}"
)
print(
    f"Check if [1, 0, 0] and [0, 1, 0] and [0, 0, 1] are valis basis: {qcheckers.check_basis([qstate([1, 0, 0]), qstate([0, 1, 0]), qstate([0, 0, 1])])}"
)

Check basis (orthonormality):

Check if | 0 > and | 1 > are valis basis: True
Check if | + > and | - > are valis basis: True
Check if [1, 0, 0] and [0, 1, 0] and [0, 0, 1] are valis basis: True


In [31]:
print(
    f"Check if [0, 0.5] and [0.5, 0] are valis basis: {qcheckers.check_basis([qstate([0, 0.5]), qstate([0.5, 0])], verbose = True)}"
)

Vectors are orthogonal: True | Vectors are normalized: False
Check if [0, 0.5] and [0.5, 0] are valis basis: False


In [12]:
display(
    Math(
        r"Check \ if \ | \Psi \rangle = \frac{1}{\sqrt{2}} [1, 1, 1, 1] \ is \ a \ qubit"
    )
)

<IPython.core.display.Math object>

In [32]:
# Check qubit
print("Check qubit:\n")
print(f"Check if | 0 > is a qubit: {qcheckers.check_qubit(qbasis.zero)}")
print(
    f"Check if | Ψ - > is a qubit: {qcheckers.check_qubit(qstate([0, 1 / sqrt(2), -1 / sqrt(2), 0]))}"
)

# Sheet 1 | Q2
display(
    Math(
        r"Check \ if \ | \Psi \rangle = \frac{1}{\sqrt{2}} [1, 1, 1, 1] \ is \ a \ qubit"
    )
)
print(
    qcheckers.check_qubit(qstate([1 / sqrt(2), 1 / sqrt(2), 1 / sqrt(2), 1 / sqrt(2)]))
)

display(
    Math(
        r"Check \ if \ | \Phi \rangle = \frac{1}{2} |00 \rangle + \frac{1}{2} |01 \rangle + \frac{1}{2} |10 \rangle + \frac{1}{2} |11 \rangle \ is \ a \ qubit"
    )
)
print(qcheckers.check_qubit(qstate([1 / 2, 1 / 2, 1 / 2, 1 / 2])))

Check qubit:

Check if | 0 > is a qubit: True
Check if | Ψ - > is a qubit: True


<IPython.core.display.Math object>

False


<IPython.core.display.Math object>

True


## Density Matrix
<a id='dmatrix'></a>


In [14]:
# Valid Density Matrix
P = density_matrix([[1 / 3, 1.0j / 3], [-1.0j / 3, 2 / 3]])
# print this 2x2 matrix in a nice way
for row in P:
    print(row)

print(f"Valid density matrix: {qcheckers.check_valid_density_matrix(P)}")
print(f"Type: {qhelpers.determine_state(P)}")

tensor([0.3333+0.0000j, 0.0000+0.3333j])
tensor([-0.0000-0.3333j, 0.6667+0.0000j])
Valid density matrix: True
Type: State is mixed


In [15]:
# Valid Density Matrix
P = density_matrix([[1 / 2, 0], [0, 1 / 2]])
# print this 2x2 matrix in a nice way
for row in P:
    print(row)

print(f"Valid density matrix: {qcheckers.check_valid_density_matrix(P)}")
print(f"Type: {qhelpers.determine_state(P)}")

tensor([0.5000+0.j, 0.0000+0.j])
tensor([0.0000+0.j, 0.5000+0.j])
Valid density matrix: True
Type: State is completely mixed


In [16]:
# Valid Density Matrix
P = density_matrix([[1 / 2, -1.0j / 5], [0, 1 / 2]])
# print this 2x2 matrix in a nice way
for row in P:
    print(row)

print(f"Valid density matrix: {qcheckers.check_valid_density_matrix(P)}")
print(f"Type: {qhelpers.determine_state(P, verbose=True)}")

tensor([0.5000+0.0000j, -0.0000-0.2000j])
tensor([0.0000+0.j, 0.5000+0.j])
Valid density matrix: False
Matrix is not Hermitian
Type: Invalid density matrix


## Finding Probability
<a id='probs'></a>

In [17]:
# Sheet 1 | Q 4

# |𝜓〉=1√3|0〉+ √23|1〉
display(
    Math(
        r"| \psi \rangle = \frac{1}{\sqrt{3}} |0\rangle + \sqrt{\frac{2}{3}} |1\rangle"
    )
)
psi = qstate([1 / sqrt(3), sqrt(2 / 3)])
print(
    f"Probability of finding | 0 > is {qhelpers.find_probability(psi, qbasis.zero):.2f}"
)

# # |𝜓〉=(1+𝑖) / √3|0〉 − 𝑖 / √3|1〉
display(
    Math(
        r"| \psi \rangle = \frac{1+i}{\sqrt{3}} |0\rangle - \frac{i}{\sqrt{3}} |1\rangle"
    )
)
psi = qstate([(1 + 1j) / sqrt(3), (-1j / sqrt(3))])
print(
    f"Probability of finding | 0 > is {qhelpers.find_probability(psi, qbasis.zero):.2f}"
)

# Sheet 1 | Q5
# |𝜓〉 = 1/2|0〉+ √3 / 2|1〉
display(Math(r"| \psi \rangle = \frac{1}{2} |0\rangle + \frac{\sqrt{3}}{2} |1\rangle"))
psi = qstate([1 / 2, sqrt(3) / 2])
print(
    f"Probability of finding | + > is {qhelpers.find_probability(psi, qbasis.plus):.2f}"
)

<IPython.core.display.Math object>

Probability of finding | 0 > is 0.33


<IPython.core.display.Math object>

Probability of finding | 0 > is 0.67


<IPython.core.display.Math object>

Probability of finding | + > is 0.93


# Quantum Gates
<a id='gates'></a>

![Qgates image](assets/qgates.png)

## Check if the gates are unitary
<a id ="unitary"><a>

In [18]:
X = qgates.X(None, gate_matrix=True)
print(f"Check if X Pauli gate is unitary: {qcheckers.check_unitary(X)}")

H = qgates.H(None, gate_matrix=True)
print(f"Check if H Hadamard gate is unitary: {qcheckers.check_unitary(H)}")

CNOT = qgates.CNOT(None, None, gate_matrix=True)
print(f"Check if CNOT gate is unitary: {qcheckers.check_unitary(CNOT)}")

# Define any arbitrary non-unitary matrix
A = qstate([[1, 2], [3, 4]])
print(f"Check if A is unitary: {qcheckers.check_unitary(A)}")

Check if X Pauli gate is unitary: True
Check if H Hadamard gate is unitary: True
Check if CNOT gate is unitary: True
Check if A is unitary: False


## Single Qubit Gates
<a id="single"><a>

In [19]:
# X Pauli Gate
print("X Pauli Gate:\n")
print(f"X | 0 >:\ng{qgates.X(qbasis.zero)}")
print(f"X | 1 >:\ng{qgates.X(qbasis.one)}")
print(f"X | + >:\ng{qgates.X(qbasis.plus)}")

# Z Pauli Gate
print("Z Pauli Gate:\n")
print(f"Z | 0 >:\ng{qgates.Z(qbasis.zero)}")
print(f"Z | 1 >:\ng{qgates.Z(qbasis.one)}")
print(f"Z | + >:\ng{qgates.Z(qbasis.plus)}")

X Pauli Gate:

X | 0 >:
gtensor([[0.+0.j],
        [1.+0.j]])
X | 1 >:
gtensor([[1.+0.j],
        [0.+0.j]])
X | + >:
gtensor([[0.7071+0.j],
        [0.7071+0.j]])
Z Pauli Gate:

Z | 0 >:
gtensor([[1.+0.j],
        [0.+0.j]])
Z | 1 >:
gtensor([[ 0.+0.j],
        [-1.+0.j]])
Z | + >:
gtensor([[ 0.7071+0.j],
        [-0.7071+0.j]])


In [20]:
# Y Pauli Gate
print("Y Pauli Gate:\n")
print(f"Y | 0 >:\ng{qgates.Y(qbasis.zero)}")
print(f"Y | 1 >:\ng{qgates.Y(qbasis.one)}")

# Hadamard Gate
print("Hadamard Gate:\n")
print(f"H | 0 >:\ng{qgates.H(qbasis.zero)}")
print(f"H | 1 >:\ng{qgates.H(qbasis.one)}")
print(f"H | + >:\ng{qgates.H(qbasis.plus)}")
print(f"H | - >:\ng{qgates.H(qbasis.minus)}")

Y Pauli Gate:

Y | 0 >:
gtensor([[0.+0.j],
        [0.+1.j]])
Y | 1 >:
gtensor([[0.-1.j],
        [0.+0.j]])
Hadamard Gate:

H | 0 >:
gtensor([[0.7071+0.j],
        [0.7071+0.j]])
H | 1 >:
gtensor([[ 0.7071+0.j],
        [-0.7071+0.j]])
H | + >:
gtensor([[1.0000+0.j],
        [0.0000+0.j]])
H | - >:
gtensor([[0.0000+0.j],
        [1.0000+0.j]])


## Multi Qubit Gates
<a id = "multi"><a>

In [21]:
print("CNOT Gate:\n")
print(f"CNOT | 0 0 >:\ng{qgates.CNOT(qbasis.zero, qbasis.zero)}")
print(f"CNOT | 0 1 >:\ng{qgates.CNOT(qbasis.zero, qbasis.one)}")
print(f"CNOT | 1 1 >:\ng{qgates.CNOT(qbasis.one, qbasis.one)}")

CNOT Gate:

CNOT | 0 0 >:
gtensor([[1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]])
CNOT | 0 1 >:
gtensor([[0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j]])
CNOT | 1 1 >:
gtensor([[0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j]])


In [22]:
print("Bell State Gate\n")
print(
    f"H | 0 0 >: (should be | Φ + >):\n{qgates.BellStateGate(qbasis.zero, qbasis.zero)}"
)
print(
    f"H | 0 1 >: (should be | Ψ + >:\n{qgates.BellStateGate(qbasis.zero, qbasis.one)}"
)
print(
    f"H | 1 0 >: (should be | Φ - >):\n{qgates.BellStateGate(qbasis.one, qbasis.zero)}"
)
print(
    f"H | 1 1 >: (should be | Ψ - >):\n{qgates.BellStateGate(qbasis.one, qbasis.one)}"
)

Bell State Gate

H | 0 0 >: (should be | Φ + >):
tensor([[0.7071+0.j],
        [0.0000+0.j],
        [0.0000+0.j],
        [0.7071+0.j]])
H | 0 1 >: (should be | Ψ + >:
tensor([[0.0000+0.j],
        [0.7071+0.j],
        [0.7071+0.j],
        [0.0000+0.j]])
H | 1 0 >: (should be | Φ - >):
tensor([[ 0.7071+0.j],
        [ 0.0000+0.j],
        [ 0.0000+0.j],
        [-0.7071+0.j]])
H | 1 1 >: (should be | Ψ - >):
tensor([[ 0.0000+0.j],
        [ 0.7071+0.j],
        [-0.7071+0.j],
        [ 0.0000+0.j]])


In [23]:
print("Ctrl Hadamard\n")
print(f"H | 0 0 >:\n{qgates.Ctrl_Hadamard(qbasis.zero, qbasis.zero)}")
print(f"H | 0 1 >:\n{qgates.Ctrl_Hadamard(qbasis.zero, qbasis.one)}")
print(f"H | 1 0 >:\n{qgates.Ctrl_Hadamard(qbasis.one, qbasis.zero)}")
print(f"H | 1 1 >:\n{qgates.Ctrl_Hadamard(qbasis.one, qbasis.one)}")

Ctrl Hadamard

H | 0 0 >:
tensor([[1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]])
H | 0 1 >:
tensor([[0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j]])
H | 1 0 >:
tensor([[0.0000+0.j],
        [0.0000+0.j],
        [0.7071+0.j],
        [0.7071+0.j]])
H | 1 1 >:
tensor([[ 0.0000+0.j],
        [ 0.0000+0.j],
        [ 0.7071+0.j],
        [-0.7071+0.j]])


In [24]:
print("Swap Gate\n")
print(f"Swap | 0 0 >:\n{qgates.swap_gate(qbasis.zero, qbasis.zero)}")
print(f"Swap | 0 1 >:\n{qgates.swap_gate(qbasis.zero, qbasis.one)}")
print(
    f"Swap | + 0 >:\n{qgates.swap_gate(qbasis.plus, qbasis.zero, format = 'ketbra')}")
print(f"Swap | 0 + >:\n{qgates.swap_gate(qbasis.zero, qbasis.plus)}")

Swap Gate

Swap | 0 0 >:
tensor([[1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j]])
Swap | 0 1 >:
tensor([[0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j]])
Swap | + 0 >:
(tensor([[1.+0.j],
        [0.+0.j]]), tensor([[0.7071+0.j],
        [0.7071+0.j]]))
Swap | 0 + >:
tensor([[0.7071+0.j],
        [0.0000+0.j],
        [0.7071+0.j],
        [0.0000+0.j]])


In [25]:
print("Ctrl Swap Gate\n")
print(
    f"Swap | 0 0 0 >:\n{qgates.Ctrl_Swap(qbasis.zero, qbasis.zero, qbasis.one)}")
print(
    f"Swap | 0 0 1 >:\n{qgates.Ctrl_Swap(qbasis.one, qbasis.zero, qbasis.one)}")

Ctrl Swap Gate

Swap | 0 0 0 >:
tensor([[0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j]])
Swap | 0 0 1 >:
tensor([[0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j]])


In [26]:
print("Toffoli Gate\n")
print(
    f"Toffoli(1, 0, 0) (# should be | 0 >):\n{qgates.Toffoli(qbasis.one, qbasis.zero, qbasis.zero)}"
)
print(
    f"Toffoli(1, 1, 0) (# should be | 1 >):\n{qgates.Toffoli(qbasis.one, qbasis.one, qbasis.zero)}"
)
print(
    f"Toffoli(0, 1, 1) (# should be | 1 >):\n{qgates.Toffoli(qbasis.zero, qbasis.one, qbasis.one)}"
)

Toffoli Gate

Toffoli(1, 0, 0) (# should be | 0 >):
tensor([[1.+0.j],
        [0.+0.j]])
Toffoli(1, 1, 0) (# should be | 1 >):
tensor([[0.+0.j],
        [1.+0.j]])
Toffoli(0, 1, 1) (# should be | 1 >):
tensor([[0.+0.j],
        [1.+0.j]])


In [28]:
# Building AND uising Tofooli (Sheet 3 | Q1)
c = qbasis.zero
print(f" | 0 > AND | 0 > :\n{qgates.Toffoli(qbasis.zero, qbasis.zero,c)}")
print(f" | 0 > AND | 1 > :\n{qgates.Toffoli(qbasis.zero, qbasis.one,c)}")
print(f" | 1 > AND | 0 > :\n{qgates.Toffoli(qbasis.one, qbasis.zero,c)}")
print(f" | 1 > AND | 1 > :\n{qgates.Toffoli(qbasis.one, qbasis.one,c)}")

# Building NOT uising Tofooli (Sheet 3 | Q2)
b = qbasis.one
c = qbasis.zero

print("-" * 50)
print(f"NOT | 0 > :\n{qgates.Toffoli(qbasis.zero, b, c)}")
print(f"NOT | 1 > :\n{qgates.Toffoli(qbasis.one, b, c)}")

 | 0 > AND | 0 > :
tensor([[1.+0.j],
        [0.+0.j]])
 | 0 > AND | 1 > :
tensor([[1.+0.j],
        [0.+0.j]])
 | 1 > AND | 0 > :
tensor([[1.+0.j],
        [0.+0.j]])
 | 1 > AND | 1 > :
tensor([[0.+0.j],
        [1.+0.j]])
--------------------------------------------------
NOT | 0 > :
tensor([[1.+0.j],
        [0.+0.j]])
NOT | 1 > :
tensor([[0.+0.j],
        [1.+0.j]])


# External Links

- [ML4SCI QML Tutorials](https://github.com/ML4SCI/QML-hands-on/tree/main/notebooks)
- [PennyLane (Quantum Machine Learning Library)](https://pennylane.ai/)